<h1>09.27(월): 웹크롤링을 이용하여 워드클라우드 구현하기</h1>
선생님: 이철현<br>
학생: 남혜빈, 이요섭<br>

웹크롤링이란??<br>
정확한 명칭으로 하면 Web Scraping이며 웹사이트에서 내가 원하는 정보만을 추출하는 것을 말합니다.<br>

<b>Step1. 파이썬에 영화 리뷰 페이지를 연결합니다.</b>

In [79]:
# requests는 파이썬으로 HTTP 소스코드를 요청할 때 가장 많이 사용되는 라이브러리입니다. --> 인터넷과 파이썬을 연결합니다.
# [0] requests를 설치합니다.
!pip install requests
# [1] requests를 사용합니다. "requests라는 기술을 가진 책을 컴퓨터가 가져올 수 있도록 읽어주세요."
import requests

In [80]:
# [2] url변수에 영화 리뷰 페이지 URL을 담습니다.
for page in range(1, 20): # --> 리뷰 페이지 0부터 20까지만 담았습니다.
  URL = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=68555&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=' +str(page)
# [3] req 변수에 http 요청해서 얻은 정보를 담습니다. --> 200으로 보아 요청은 성공입니다.
req = requests.get(url)

# [4] html 변수에 url의 html 정보를 저장합니다. 
html = req.text

**https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=68555&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1

?code = 숫자 --> 영화마다 다른 숫자를 가집니다.<br>
$page = 숫자 --> 평점 페이지를 말합니다. 가장 마지막페이지까지 모두 크롤링을 하려면 반복문을 이용하면 됩니다.

<b>Step2 영화 리뷰 페이지에서 필요한 정보만 추출합니다.</b>

<br>
requests는 html을 python이 이해하는 객체 구조로 만들어주지는 못합니다. req.text는 python의 문자열 객체를 반환하기에 정보를 추출하기는 어렵습니다.

BeautifulSoup은 html 코드를 python이 이해하는 객체 구조로 변환하는 parsing을 맡고 있고, 이 라이브러리를 이용하여 우리는 제대로 된 '의미있는' 정보만을 추출해 낼 수 있습니다.

In [81]:
# [0] bs4라이브러리를 설치합니다.
# [0](1) BeautifulSoup을 통해 설치하는 방법
# from bs4 import BeautifulSoup

# [0](2) bs4를 직접 설치하는 방법
!pip install bs4
# [1] bs4 라이브러리를 사용합니다.
from bs4 import BeautifulSoup # --> 이제 html에서 필요한 부분을 사용할 수 있습니다.

In [82]:
# [2] soup 객체에서 원하는 정보를 찾아냅시다. --> html.parser: 구문 분석 하는 친구!
soup = BeautifulSoup(html, 'html.parser')

review_sentence = soup.select('body > div > div > div > ul > li > div > p > span')

In [88]:
# [3] 원하는 문장을 출력하는 함수
def get_sentence(data): # 이런 기능을 가진 함수를 만들 거에요!

# -------------------기능-------------------------
  text = str(data) # 들어온 데이터를 str=문자열 형태로 만들어 줍니다.

  start = text.find('">') + len('">') # 시작 부분을 찾습니다.
  end = text.find("</span>") # 끝 부분을 찾습니다.

  review_sentence = text[start:end] # 시작 부분과 끝 부분 사이에 있는 문장을 뽑습니다.

# ------------------결과 배출-----------------------
  return review_sentence # 뽑은 문장을 돌려줍니다.

data_array = []

for i in range(len(sentence)):
  sent = get_sentence(sentence[i]).replace('\t',"")
  sent = sent.replace('\r',"")
  sent = sent.replace('\n',"")
  data_array.append(sent)

#  for i in range(len(data_array)):
#    print(data_array[i])

cnt = 1
for i in range(len(data_array)):
  print(str(cnt)+"번째 댓글: "+data_array[i])
  cnt+=1

1번째 댓글: 
2번째 댓글: 배우들의 리액션 연기가 다한 영화ㅋㅋ배우들 정말대단하네요 cg라서 연기하기가 더 어려웠을텐데 리액션 연기 정말 대단ㅋㅋ 
3번째 댓글: <span class="_unfold_ment" id="_unfold_ment2"><a data-src="인간미 넘치는 로봇 영화의 표본이랄까. 한 두번이 아닌 여러번 봐도 재밌는 영화. 오히려 볼 때마다 스토리의 깊은 면을 볼 수 있는 것 같음. 아들인 맥스 켄튼을 만나고 영원히 철없을 것 같은 아빠 찰리 켄틀이 서서히 성장하고 제대로 된 아버지로 탈바꿈하는 모습이 보기 좋았고 스파링 로봇 아톰으로서의 모습이 아닌 복서 아톰으로서의 모습을 잘 표현했다고 생각함. 각본이 고철이라고들 말하는데 뻔한 극적 결말보다 현실적이고 훈훈한 열린 결말이 인상적이었고 복서 시절의 찰리의 패배는 찰리에게 비관적이었지만 아톰과 함께 싸운 마지막 경기에서의 패배는 시민들의 챔피언으로 남는다는 것이 정말 인상 깊었음. 적절한 타이밍에 쓰인 OST도 신의 한 수. " href="javascript:void(0);" onclick="unfoldPointMent(this);">인간미 넘치는 로봇 영화의 표본이랄까. 한 두번이 아닌 여러번 봐도 재밌는 영화. 오히려 볼 때마다 스토리의 깊은 면을 볼 수 있는 것 같음. 아들인 맥스 켄튼을 만나고 영원히 철없을 것 같은 아빠 찰리 켄틀이 서서...</a>
4번째 댓글: 아들이랑 같이봤어요! 재밌습니다 
5번째 댓글: 2020년인데 로봇끼리 복싱하는 거 어디가면 볼 수 있는건가요? 
6번째 댓글: 그때 극장에서 봤었는데 ㅋㅋ 9년만에 봐도 재밌음 
7번째 댓글: 초6때 엄마랑 오빠랑 영화관가서 같이본기억이 아직도 생생 지금봐도 감동 ㅠㅠ그때도 남자애존잘이여서 인터넷으로바로찾아봤는데 지금많이컷겠다 
8번째 댓글: 처음 봤던 그때가 그립다.... 보고 진짜 감동받았는데 오랫만에 봐도 감동이네요 2나오면 극장에서 볼텐대 
9번째 댓글: 몇년을 지나 다시 봐도 너무 감동있고 재밌는...